In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from fl_g13.editing.sparseSGDM import SparseSGDM
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR

import flwr
from flwr.simulation import run_simulation
from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app

print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

2025-05-09 21:35:25.026 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


Flower 1.17.0 / PyTorch 2.6.0+cu124


In [3]:
import os
import urllib.request

# TODO: move in a "make client dependencies function"
def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

download_if_not_exists(
    "vision_transformer.py",
    "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py"
)

download_if_not_exists(
    "utils.py",
    "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py"
)

'vision_transformer.py' already exists.
'utils.py' already exists.


In [4]:
# Settings
CHECKPOINT_DIR = "/home/massimiliano/Projects/fl-g13/checkpoints"

# Model hyper-parameters
head_layers=3
head_hidden_size=512
dropout_rate=0.0
unfreeze_blocks=1

# Training hyper-parameters
starting_lr = 1e-3
momentum = 0.9
weight_decay=1e-5
T_max=8
eta_min=1e-5

# Federated Training setting
batch_size = 64
local_epochs = 1
number_of_rounds = 5
fraction_fit = 1
fraction_evaluate = 0.1
number_of_clients = 2
min_num_clients = 2
partition_type = "iid" # or "shard"
num_shards_per_partition = 6
use_wandb = False

# Device settings
device = "cuda" if torch.cuda.is_available() else "cpu"
backend_config = {
    "client_resources": {
        "num_cpus": 1, 
        "num_gpus": 0
    }
}

# When running on GPU, assign an entire GPU for each client
if device == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}

    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

print(f"Training on {device}")

Training on cuda


In [5]:
# Model
model = BaseDino(
    head_layers=head_layers, 
    head_hidden_size=head_hidden_size, 
    dropout_rate=dropout_rate, 
    unfreeze_blocks=unfreeze_blocks
    )
model.to(device)

mask = [torch.ones_like(p, device=p.device) for p in model.parameters()] # Must be done AFTER the model is moved to CUDA
optimizer = SparseSGDM(
    model.parameters(),
    mask=mask,
    lr=starting_lr,
    momentum=momentum,
    weight_decay=weight_decay
    )
scheduler = CosineAnnealingLR(
    optimizer=optimizer, 
    T_max=T_max, 
    eta_min=eta_min
    )
criterion = CrossEntropyLoss()

client_app = get_client_app(
    partition_type=partition_type, 
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    local_epochs=local_epochs,
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    device=device, 
)
server_app = get_server_app(
    checkpoint_dir=CHECKPOINT_DIR,
    model_class=model.__class__,
    model_config=model.get_config(), 
    optimizer=optimizer, 
    criterion=criterion, 
    scheduler=scheduler,
    device=device, 
    num_rounds=number_of_rounds, 
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_num_clients,
    min_evaluate_clients=min_num_clients,
    min_available_clients=number_of_clients,
    use_wandb=False,
    wandb_config=None,
)

Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


🔍 Loading checkpoint from /home/massimiliano/Projects/fl-g13/checkpoints/FL_BaseDino_epoch_7.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 1, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /home/massimiliano/Projects/fl-g13/checkpoints/FL_BaseDino_epoch_7.pth, resuming at epoch 8


In [ ]:
run_simulation(
    client_app=client_app,
    server_app=server_app,
    num_supernodes=number_of_clients,
    backend_config=backend_config,
)

[Server] Server on device: cuda:0
[Server] CUDA available in client: True


INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Server Eval Round 0] Model device: cuda:0
[Server Eval Round 0] CUDA available in server eval: True


Eval progress: 100%|██████████| 313/313 [00:28<00:00, 11.16batch/s]
INFO :      initial parameters (loss, other metrics): 4.605169776148689, {'centralized_accuracy': 0.01}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


[Round 0] Centralized Evaluation - Loss: 4.6052, Metrics: {'centralized_accuracy': 0.01}


...

Eval progress: 100%|██████████| 313/313 [00:23<00:00, 13.49batch/s]
INFO :      fit progress: (3, 4.605169776148689, {'centralized_accuracy': 0.01}, 493.53320942100254)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


[Round 3] Centralized Evaluation - Loss: 4.6052, Metrics: {'centralized_accuracy': 0.01}
(ClientAppActor pid=222133) [Client] Client on device: cuda:0
(ClientAppActor pid=222133) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 79/79 [00:12<00:00,  6.32batch/s]


(ClientAppActor pid=222133) [Client] Client on device: cuda:0
(ClientAppActor pid=222133) [Client] CUDA available in client: True


Eval progress:  13%|█▎        | 10/79 [00:01<00:11,  5.90batch/s]
